In [1]:
import pandas as pd
import numpy as np
import math
import time

## Abertura e Tratamento dos dados

### Bases utilizadas: 
  - Mortalidade
  - Localidades (regiões, estados, munic., etc)

Precisamos armazenar temporariamente as localizações para poder tratar os locais referidos na base de mortalidade.

A base de mortalidade passou por um pre-processamento antes de ser lida. No novo arquivo, todos os registros de mortalidade tiveram a localidade corrigida (local de 6 digitos convertidos para 7)

In [2]:
# Ler dos arquivos
df = pd.read_csv('/home/modesto/Documentos/OPAS/consolidacao_mortalidades/dados/mortalities_news.csv', names=["ano", "ano_sem", "semana", "contexto", "local_oco", "local_res", "idade", "sexo", "raca", "tipo"])
df_localidades = pd.read_csv('/home/modesto/Documentos/OPAS/consolidacao_mortalidades/dados_antigos/locations.csv', names=["n", "pai", "id", "nome", "sigla"])# Armazena dos municípios em uma lista (serie) 

483     110002
484     110013
485     110026
486     110040
487     110045
         ...  
6048    521305
6049    521560
6050    521945
6051    522230
6052    530010
Name: id, Length: 5570, dtype: int64

In [3]:
# Armazena dos municípios em uma lista (serie) 
lista_mun = df_localidades[df_localidades['n'] == 4]['id']
lista_mun = lista_mun.apply(lambda x: x // 10)
lista_mun

483     110002
484     110013
485     110026
486     110040
487     110045
         ...  
6048    521305
6049    521560
6050    521945
6051    522230
6052    530010
Name: id, Length: 5570, dtype: int64

In [4]:
df

,ano,ano_sem,semana,contexto,local_oco,local_res,idade,sexo,raca,tipo
0,2000,NaN,NaN,A09,3169703,3138351,306.0,2,NaN,2
1,2000,NaN,NaN,A162,3106200,3118601,436.0,2,NaN,2
2,2000,NaN,NaN,A162,3106200,3118601,449.0,1,NaN,2
3,2000,NaN,NaN,A183,3106200,3146107,423.0,1,NaN,2
4,2000,NaN,NaN,A418,3106200,3146107,469.0,2,NaN,2
...,...,...,...,...,...,...,...,...,...,...
22564820,2019,2020.0,1.0,Y834,3304557,3304557,480.0,1,1.0,2
22564821,2019,2020.0,1.0,Y834,3304557,3304557,498.0,2,1.0,2
22564822,2019,2020.0,1.0,Y839,3300456,3300456,473.0,2,4.0,2
22564823,2019,2020.0,1.0,Y848,3131307,3113404,488.0,1,4.0,2


## Tratando dados vazios

Os campos idade, raca e investigacao possuem dados vazios.

- Para tratar o campo idade - adicionar o valor 1000 (será substituido posteriormente)
- Para tratar o campo raca - adicionar o valor 9 que corresponde à raça 'ignorada'
- Para tratar o campo investigacao - remover essa coluna (nao será utilizada)

In [5]:
df.isnull().sum()

ano                0
ano_sem          499
semana           499
contexto           0
local_oco          0
local_res          0
idade           3180
sexo               0
raca         1565610
tipo               0
dtype: int64

In [6]:
# Atualizar o campo idade
df.update(df['idade'].fillna(1000))
df.update(df['ano_sem'].fillna(1000))
df.update(df['semana'].fillna(1000))

In [7]:
# Atualizar o campo raca
df.update(df['raca'].fillna(9))

In [8]:
df.isnull().sum()

ano          0
ano_sem      0
semana       0
contexto     0
local_oco    0
local_res    0
idade        0
sexo         0
raca         0
tipo         0
dtype: int64

In [9]:
# Converter os campos idade e raca para int
df.loc[:,'ano_sem'] = df.loc[:,'ano_sem'].values.astype(np.int64)
df.loc[:,'semana'] = df.loc[:,'semana'].values.astype(np.int64)
df.loc[:,'idade'] = df.loc[:,'idade'].values.astype(np.int64)
df.loc[:,'raca'] = df.loc[:,'raca'].values.astype(np.int64)
df

,ano,ano_sem,semana,contexto,local_oco,local_res,idade,sexo,raca,tipo
0,2000,1000,1000,A09,3169703,3138351,306,2,9,2
1,2000,1000,1000,A162,3106200,3118601,436,2,9,2
2,2000,1000,1000,A162,3106200,3118601,449,1,9,2
3,2000,1000,1000,A183,3106200,3146107,423,1,9,2
4,2000,1000,1000,A418,3106200,3146107,469,2,9,2
...,...,...,...,...,...,...,...,...,...,...
22564820,2019,2020,1,Y834,3304557,3304557,480,1,1,2
22564821,2019,2020,1,Y834,3304557,3304557,498,2,1,2
22564822,2019,2020,1,Y839,3300456,3300456,473,2,4,2
22564823,2019,2020,1,Y848,3131307,3113404,488,1,4,2


## Reformulando o campo: CONTEXTO

O campo contexto corresponde aos CID10 dos agravos e não possui um padrão de codificação. A padronização a ser utilizada segue o seguinte exemplo:

    A00 vira 'A' e 10000
    A00.0 vira 'A' e 10010
    A00.1 vira 'A' e 10011
    ...
    A00.9 vira 'A' e 10019
    A01 vira 'A' e 10100
    A01.0 vira 'A' e 10110
    A01.1 vira 'A' e 10111
    ...
    A01.9 vira 'A' e 10119
    A99.0 vira 'A' e 19910
    A99.1 vira 'A' e 19911
    ...
    A99.9 vira 'A' e 19919

Isso será util na hora de classificar (e comparar) as doenças. Exemplo:
- Para Cólera com CID A00:

    char == 'A' and digits >= 10000 and digits <= 10019

- Para Febre Tifóide com CID A01.0:

    char == 'A' and digits == 10110

É necessário realizar o mapeamento de cada CID 10 para os novos códigos.

In [10]:
# Criação do método para converter contexto para o padrão de código mencionado
def converter_contexto(contexto):
    char = contexto[0]
    numeros = contexto.split(char)[1]
    
    final = char+'1'
    
    if len(numeros) == 1:
        final = final+numeros+'000'
    elif len(numeros) == 2:
        final = final+numeros+'00'
    elif len(numeros) == 3:
        final = final+numeros[0]+numeros[1]+'1'+numeros[2]
    return final

In [11]:
# Aplicando o método
df.loc[:,'contexto'] = df.loc[:,'contexto'].apply(converter_contexto)
df

,ano,ano_sem,semana,contexto,local_oco,local_res,idade,sexo,raca,tipo
0,2000,1000,1000,A10900,3169703,3138351,306,2,9,2
1,2000,1000,1000,A11612,3106200,3118601,436,2,9,2
2,2000,1000,1000,A11612,3106200,3118601,449,1,9,2
3,2000,1000,1000,A11813,3106200,3146107,423,1,9,2
4,2000,1000,1000,A14118,3106200,3146107,469,2,9,2
...,...,...,...,...,...,...,...,...,...,...
22564820,2019,2020,1,Y18314,3304557,3304557,480,1,1,2
22564821,2019,2020,1,Y18314,3304557,3304557,498,2,1,2
22564822,2019,2020,1,Y18319,3300456,3300456,473,2,4,2
22564823,2019,2020,1,Y18418,3131307,3113404,488,1,4,2


### Mapa dos agravos e códigos utilizados

É necessário criar uma relação entre CIDs e os campos de local de residência e ocorrência. Para isso, utilizou-se o seguinte padrão:

- 10100 para agravo A
- 10200 para agravo B

Onde: 
- Os três primeiros dígitos do código mapeam o CID 10
- Os dois últimos dígitos do código mapeam se o indicador é de residência (00) ou ocorrência (01)

In [12]:
# Criação do método para comparar os agravos e gerar novo codigo para o contexto
def gerar_novo_contexto(contexto, regiao):
    char = contexto[0]
    numeros = contexto.split(char)[1]
    numeros = int(numeros)
    string_regiao = str(regiao)
    numero_da_regiao = string_regiao[0]    
    
    if char == 'A' and numeros == 10000:
        return 100100 
    elif char == 'A' and numeros == 12000:
        return 101900 
    elif char == 'A' and numeros == 18000:
        return 101800 
    elif char == 'A' and numeros == 10511:
        return 100000 
    elif char == 'A' and numeros == 19213:
        return 101300 
    elif char == 'A' and numeros == 18200:
        return 102000 
    elif char == 'P' and numeros == 13500:
        return 102100 
    elif char == 'B' and numeros == 10600:
        return 102300 
    elif char == 'B' and numeros == 10500:
        return 102200 
    elif char == 'A' and numeros == 19000:
        return 110000 
    elif char == 'A' and numeros == 19210:
        return 110200 
    elif char == 'A' and numeros == 19200:
        return 110200 
    elif char == 'U' and numeros == 10600:
        return 110100 
    elif char == 'A' and numeros == 19500:
        return 101200 
    elif char == 'A' and numeros == 17719:
        return 101400
    elif char == 'A' and numeros == 19818:
        return 101500
    elif char == 'B' and numeros == 15400 and numero_da_regiao == '1':
        return 411000
    elif char == 'B' and numeros == 15400 and numero_da_regiao != '1':
        return 101700
    elif char == 'A' and numeros == 18019:
        return 102400
    elif char == 'X' and numeros == 12900:
        return 300100
    elif char == 'A' and numeros == 13719:
        return 200000
    elif char == 'A' and numeros == 13619:
        return 200100
    elif char == 'B' and numeros == 15711:
        return 200200
    elif char == 'B' and numeros == 16519:
        return 410300
    elif char == 'A' and numeros == 10110:
        return 200500
    elif char == 'A' and numeros == 13019:
        return 410400
    elif char == 'B' and numeros == 11900:
        return 410500
    elif char == 'T' and numeros == 16519:
        return 410700
    elif char == 'B' and numeros == 15511:
        return 410800
    elif char == 'B' and numeros == 15510:
        return 410900
    elif char == 'A' and numeros == 12719:
        return 300300
    elif char == 'A' and numeros == 13500:
        return 300400
    elif char == 'A' and numeros == 13300:
        return 300500
    elif char == 'A' and numeros == 11619:
        return 411500
    elif char == 'B' and numeros == 10100:
        return 200600
    elif char == 'A' and numeros == 13900:
        return 200400
    elif char == 'W' and numeros == 16400:
        return 300200
    elif char == 'A' and numeros == 13400:
        return 411200
    elif char == 'D' and numeros == 13912:
        return 411200
    elif char == 'E' and numeros == 12310:
        return 411200
    elif char == 'F' and numeros == 15300:
        return 411200
    elif char == 'M' and numeros == 18310:
        return 411200
    elif char == 'X' and numeros >= 18500 and numeros <= 19900:
        return 300800
    elif char == 'Y' and numeros >= 10000 and numeros <= 10900:
        return 300800
    elif char == 'Y' and numeros >= 13500 and numeros <= 13600:
        return 300800
    elif char == 'X' and numeros >= 16000 and numeros <= 18400:
        return 300700
    elif char == 'V' and numeros >= 10100 and numeros <= 19900:
        return 411700
    elif char == 'B' and numeros >= 12000 and numeros <= 12400:
        return 411200
    elif char == 'O' and numeros >= 10000 and numeros <= 16500:
        return 411200
    elif char == 'O' and numeros >= 16800 and numeros <= 19500:
        return 411200
    elif char == 'O' and numeros >= 19800 and numeros <= 19900:
        return 411200
    else:
        return 1

In [13]:
df.loc[:,'contexto_00'] = int(1)
df.loc[:,'contexto_01'] = int(1)

In [14]:
# Aplicando o método
df.loc[:,'contexto_00'] = df.apply(lambda x: gerar_novo_contexto(x['contexto'], x['local_res']), axis=1)

In [15]:
df.loc[:,'contexto_01'] = df.apply(lambda x: gerar_novo_contexto(x['contexto'], x['local_oco']), axis=1)

In [16]:
df.to_csv('csv_pos_ajustes_contextos.csv')

# Recomeçando aqui!!!!!!

In [1]:
import pandas as pd
import numpy as np
import math
import time

In [2]:
df_teste = pd.read_csv('/home/modesto/Documentos/OPAS/csv_pos_ajustes_contextos.csv')

In [3]:
df_localidades = pd.read_csv('/home/modesto/Documentos/OPAS/consolidacao_mortalidades/dados_antigos/locations.csv', names=["n", "pai", "id", "nome", "sigla"])# Armazena dos municípios em uma lista (serie) 

In [4]:
df_teste = df_teste.drop('Unnamed: 0', axis=1)
# df

In [5]:
# def gerar_novo_contexto_malaria(regiao):
#     string_regiao = str(regiao)
#     numero_da_regiao = string_regiao[0]
    
#     if numero_da_regiao == '1':
#         return 411000
#     else:
#         return 101700

In [6]:
#df.loc[:,'contexto_00'] = df.loc[:,'contexto'].apply(gerar_novo_contexto)

In [7]:
#df.loc[:,'contexto_01'] = df.loc[:,'contexto'].apply(gerar_novo_contexto)

In [8]:
# df[df['contexto'] == 'B15400']

O método anterior ainda NÃO faz distinção dos indicadores para residência e ocorrência. Isso será feito posteriormente.

## Reformulando os campos: IDADE, SEXO, RACA e TIPO

O campo idade possui um código próprio que segue o seguinte padrão:

    se o primeiro digito for 1, a idade é em minutos
    se for 2, a idade é em horas
    se 3, a idade é em mês
    se 4, a idade em anos
    se 5, a pessoa tem mais de 100 anos
    os próximos dois dígitos indicam a idade de fato
    Ex.: 435 - pessoa de 35 anos
    
O campo sexo possui os seguintes padrão:

    1- homem
    2- mulher
    9- ignorado
    
O campo raca possui os seguintes padrão:

    1- raça caucasian
    2- raça african
    3- raça asian
    4- raça brown
    5- raça native
    9- raça ignorada
    
O campo tipo possui os seguintes padrão:

    1- óbito nao fetal
    2- óbito fetal
    
É necessário transformar todos esses campos em dummy (tabela binária para representar cada campo). Além disso, é necessário combinar os campos idade e sexo, para isso, adotou-se o seguinte padrão:
    
    O primeiro campo é a letra do sexo (M, F ou I)
    Os próximos dois digitos correspondem ao intervalo da idade máxima do registro (04 até 4 anos, 09 até 9 anos, 14 até 14 anos e etc)    

In [9]:
# Separando uma amostra de 2.000 registros para teste
df_teste = df_teste.iloc[5000000:6000000,]
df_teste

,ano,ano_sem,semana,contexto,local_oco,local_res,idade,sexo,raca,tipo,contexto_00,contexto_01
4000000,2004,2004,16,R19900,3171204,3162955,452,1,4,2,1,1
4000001,2004,2004,16,R19900,3202405,3202405,476,1,4,2,1,1
4000002,2004,2004,16,R19900,3300209,3300209,430,1,9,2,1,1
4000003,2004,2004,16,R19900,3300209,3300209,435,1,2,2,1,1
4000004,2004,2004,16,R19900,3300209,3300209,475,1,1,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,2005,2005,15,R19900,1302603,1302603,456,2,4,2,1,1
4999996,2005,2005,15,R19900,1302603,1302603,463,1,4,2,1,1
4999997,2005,2005,15,R19900,1302603,1302603,477,2,1,2,1,1
4999998,2005,2005,15,R19900,1302603,1302603,486,2,4,2,1,1


In [10]:
# Criando o mapa de dummys de idade e sexo para o sexo Masculino
df_teste.loc[:, 'M04'] = int(0)
df_teste.loc[:, 'M09'] = int(0) 
df_teste.loc[:, 'M14'] = int(0) 
df_teste.loc[:, 'M19'] = int(0) 
df_teste.loc[:, 'M24'] = int(0) 
df_teste.loc[:, 'M29'] = int(0) 
df_teste.loc[:, 'M34'] = int(0) 
df_teste.loc[:, 'M39'] = int(0) 
df_teste.loc[:, 'M44'] = int(0) 
df_teste.loc[:, 'M49'] = int(0) 
df_teste.loc[:, 'M54'] = int(0) 
df_teste.loc[:, 'M59'] = int(0) 
df_teste.loc[:, 'M64'] = int(0) 
df_teste.loc[:, 'M69'] = int(0) 
df_teste.loc[:, 'M74'] = int(0) 
df_teste.loc[:, 'M79'] = int(0)
df_teste.loc[:, 'M80'] = int(0)

In [11]:
# Criando o mapa de dummys de idade e sexo para o sexo Feminino
df_teste.loc[:, 'F04'] = int(0) 
df_teste.loc[:, 'F09'] = int(0) 
df_teste.loc[:, 'F14'] = int(0) 
df_teste.loc[:, 'F19'] = int(0) 
df_teste.loc[:, 'F24'] = int(0) 
df_teste.loc[:, 'F29'] = int(0) 
df_teste.loc[:, 'F34'] = int(0) 
df_teste.loc[:, 'F39'] = int(0) 
df_teste.loc[:, 'F44'] = int(0) 
df_teste.loc[:, 'F49'] = int(0) 
df_teste.loc[:, 'F54'] = int(0) 
df_teste.loc[:, 'F59'] = int(0) 
df_teste.loc[:, 'F64'] = int(0) 
df_teste.loc[:, 'F69'] = int(0) 
df_teste.loc[:, 'F74'] = int(0) 
df_teste.loc[:, 'F79'] = int(0)
df_teste.loc[:, 'F80'] = int(0)

In [12]:
# Criando o mapa de dummys de idade e sexo para o sexo Ignorado
df_teste.loc[:, 'I04'] = int(0) 
df_teste.loc[:, 'I09'] = int(0) 
df_teste.loc[:, 'I14'] = int(0) 
df_teste.loc[:, 'I19'] = int(0) 
df_teste.loc[:, 'I24'] = int(0) 
df_teste.loc[:, 'I29'] = int(0) 
df_teste.loc[:, 'I34'] = int(0) 
df_teste.loc[:, 'I39'] = int(0) 
df_teste.loc[:, 'I44'] = int(0) 
df_teste.loc[:, 'I49'] = int(0) 
df_teste.loc[:, 'I54'] = int(0) 
df_teste.loc[:, 'I59'] = int(0) 
df_teste.loc[:, 'I64'] = int(0) 
df_teste.loc[:, 'I69'] = int(0) 
df_teste.loc[:, 'I74'] = int(0) 
df_teste.loc[:, 'I79'] = int(0)
df_teste.loc[:, 'I80'] = int(0)
df_teste.loc[:, 'SIG'] = int(0)

In [13]:
# Criando o mapa de dummys de raca
df_teste.loc[:, 'R01'] = int(0)
df_teste.loc[:, 'R02'] = int(0)
df_teste.loc[:, 'R03'] = int(0)
df_teste.loc[:, 'R04'] = int(0)
df_teste.loc[:, 'R05'] = int(0)
df_teste.loc[:, 'RIG'] = int(0)

# Criando o mapa de dummys de tipo
df_teste.loc[:, 'T01'] = int(0)
df_teste.loc[:, 'T02'] = int(0)
df_teste.loc[:, 'TIG'] = int(0)

# Criando o mapa de dummys o campo investigacao (novo campo)
df_teste.loc[:, 'I01'] = int(0)
df_teste.loc[:, 'I02'] = int(0)
df_teste.loc[:, 'IIG'] = int(0)

In [14]:
# df_teste

In [15]:
# Criação do método que identifica sexo e idade e realiza o dummy 
def identificar_idade(idade, sexo):
    if (idade == 1000):
        return 'SIG'
    elif (idade <= 404):
        return identificar_sexo(sexo, '04')
    elif (idade > 404) and (idade <= 409):
        return identificar_sexo(sexo, '09')
    elif (idade > 409) and (idade <= 414):
        return identificar_sexo(sexo, '14')
    elif (idade > 414) and (idade <= 419):
        return identificar_sexo(sexo, '19')
    elif (idade > 419) and (idade <= 424):
        return identificar_sexo(sexo, '24')
    elif (idade > 424) and (idade <= 429):
        return identificar_sexo(sexo, '29')
    elif (idade > 429) and (idade <= 434):
        return identificar_sexo(sexo, '34')
    elif (idade > 434) and (idade <= 439):
        return identificar_sexo(sexo, '39')
    elif (idade > 439) and (idade <= 444):
        return identificar_sexo(sexo, '44')
    elif (idade > 444) and (idade <= 449):
        return identificar_sexo(sexo, '49')
    elif (idade > 449) and (idade <= 454):
        return identificar_sexo(sexo, '54')
    elif (idade > 454) and (idade <= 459):
        return identificar_sexo(sexo, '59')
    elif (idade > 459) and (idade <= 464):
        return identificar_sexo(sexo, '64')
    elif (idade > 464) and (idade <= 469):
        return identificar_sexo(sexo, '69')
    elif (idade > 469) and (idade <= 474):
        return identificar_sexo(sexo, '74')
    elif (idade > 474) and (idade <= 479):
        return identificar_sexo(sexo, '79')
    else:
        return identificar_sexo(sexo, '80')

def identificar_sexo(sexo, idade):
    if (sexo == 1):
        return 'M'+idade
    elif (sexo == 2):
        return 'F'+idade
    else:
        return 'I'+idade

In [16]:
# Criação do método que identifica raca, tipo, investigacao realiza o dummy 
def identificar_raca(raca):
    if (raca == 1):
        return 'R01'
    elif (raca == 2):
        return 'R02'
    elif (raca == 3):
        return 'R03'
    elif (raca == 4):
        return 'R04'
    elif (raca == 5):
        return 'R05'
    else:
        return 'RIG'

def identificar_tipo(tipo):
    if (tipo == 1):
        return 'T01'
    elif (tipo == 2):
        return 'T02'
    else:
        return 'TIG'

def identificar_investigacao(investicacao):
    if (investicacao == 1):
        return 'I01'
    elif (investicacao == 2):
        return 'I02'
    else:
        return 'IIG'

In [17]:
dados = []
colunas_df_teste = df_teste.columns

def criar_df(dados):
    return pd.DataFrame(dados, columns=colunas_df_teste)
       
def carregar_nova_linha(nova_linha, coluna_idade, coluna_raca, coluna_tipo, coluna_investigacao):
    nova_linha[coluna_idade] = int(1)
    nova_linha[coluna_raca] = int(1)
    nova_linha[coluna_tipo] = int(1)
    nova_linha[coluna_investigacao] = int(1)
    aux = []
    for col in colunas_df_teste:
        aux.append(nova_linha[col])
    dados.append(aux)

In [18]:
# Execução dos métodos de dummy
ini = time.time()

for index, row in df_teste.iterrows():
    if index % 100000 == 0:
        print(f'Index: {index}')
        
    coluna_idade = identificar_idade(row['idade'], row['sexo'])
    coluna_raca = identificar_raca(row['raca'])
    coluna_tipo = identificar_tipo(row['tipo'])
    coluna_investigacao = identificar_investigacao(0)
    
    nova_linha = carregar_nova_linha(row, coluna_idade, coluna_raca, coluna_tipo, coluna_investigacao)

fim = time.time()
print("Método de carregar linha: ", fim-ini)

ini = time.time()
arr = np.array(dados)
fim = time.time()
print("Método de criar np.array: ", fim-ini)
      
print(f'Inicio da criacao do dataframe')

ini = time.time()
df_news = criar_df(arr)
fim = time.time()
print("Método de criar novo DF: ", fim-ini)

Index: 4000000
Index: 4100000
Index: 4200000
Index: 4300000
Index: 4400000
Index: 4500000
Index: 4600000
Index: 4700000
Index: 4800000
Index: 4900000
Método de carregar linha:  505.25346970558167
Método de criar np.array:  13.55691409111023
Inicio da criacao do dataframe
Método de criar novo DF:  3.079327344894409


In [19]:
# df_news

In [20]:
# Remoção dos campos idade, sexo, raca e tipo (não serão mais utilizados)
df_news = df_news.drop(columns='idade', axis=0)
df_news = df_news.drop(columns='sexo', axis=0)
df_news = df_news.drop(columns='raca', axis=0)
df_news = df_news.drop(columns='tipo', axis=0)
df_news = df_news.drop(columns='contexto', axis=0)

In [21]:
# Gerando um dicionario (municipio: regiao_saude)
df_temp = df_localidades[df_localidades['n']==4]
dict_regiao_saude = dict([(i,int(c)) for i,c in zip(df_temp['id'], df_temp['pai'])])

In [22]:
def buscar_regiao_saude(municipio):
    municipio = int(municipio)
    if municipio == 1:
        return 1
    return dict_regiao_saude[municipio]

def encontrar_regiao(municipio):
    municipio_string = str(municipio)
    return municipio_string[0]

def encontrar_estado(municipio):
    municipio_string = str(municipio)
    return municipio_string[0:2]

In [23]:
# Adição do campo que serão utlizados na consolidação
df_news.loc[:, 'total_casos'] = 1
df_news.loc[:, 'pais'] = 76

In [24]:
df_news.loc[:, 'regiao_00'] = df_news.loc[:, 'local_res'].apply(encontrar_regiao)
df_news.loc[:, 'regiao_01'] = df_news.loc[:, 'local_oco'].apply(encontrar_regiao)

In [25]:
df_news.loc[:, 'estado_00'] = df_news.loc[:, 'local_res'].apply(encontrar_estado)
df_news.loc[:, 'estado_01'] = df_news.loc[:, 'local_oco'].apply(encontrar_estado)

In [26]:
df_news.loc[:, 'regiao_saude_00'] = df_news.loc[:,'local_res'].apply(buscar_regiao_saude)
df_news.loc[:, 'regiao_saude_01'] = df_news.loc[:,'local_oco'].apply(buscar_regiao_saude)

In [27]:
# df_news

## Realizando o agrupamento de ANO, CONTEXTO e LOCAIS

Os agrupamentos devem ser realizados considerando as seguintes condições:

- Contexto XXX00 para casos de residência e XXX01 para casos de ocorrência
- O campo ano deve ser agrupado como ano e como semana epidemológica
- O campo local (res e oco) devem ser segmentados em país, região, estados, região de saúde e município

Estratégia para realizar o agrupamento de toda a base:

- Separar a cada 3.1kk de linhas

- Lembrar de:

    - Remover antes da consolidação por CONTEXTO os valores 1 (CIDS fora da planilha)
    - Remover antes da consolidação por SEMANA as linhas com valor igual a 1000
    - Remover antes de consolidação por LOCALIDADE as linhas com localidade igual a 1

    - Gerar um arquivo pra ANO e outro pra SEMANA 

In [28]:
columns = ['total_casos', 'M04', 'M09', 'M14', 'M19', 'M24', 'M29',
       'M34', 'M39', 'M44', 'M49', 'M54', 'M59', 'M64', 'M69', 'M74', 'M79',
       'M80', 'F04', 'F09', 'F14', 'F19', 'F24', 'F29', 'F34', 'F39', 'F44',
       'F49', 'F54', 'F59', 'F64', 'F69', 'F74', 'F79', 'F80', 'I04', 'I09',
       'I14', 'I19', 'I24', 'I29', 'I34', 'I39', 'I44', 'I49', 'I54', 'I59',
       'I64', 'I69', 'I74', 'I79', 'I80', 'SIG', 'R01', 'R02', 'R03', 'R04',
       'R05', 'RIG', 'T01', 'T02', 'TIG', 'I01', 'I02', 'IIG',]

In [29]:
for column in columns:
    df_news.loc[:, column] = df_news.loc[:, column].values.astype(np.int32)

### Filtro ANO, LOCAL_RES, CONTEXTO_00

In [30]:
col1 = ['contexto', 'pais', 'ano', 'total_casos', 'M04', 'M09', 'M14', 'M19',
       'M24', 'M29', 'M34', 'M39', 'M44', 'M49', 'M54', 'M59', 'M64', 'M69',
       'M74', 'M79', 'M80', 'F04', 'F09', 'F14', 'F19', 'F24', 'F29', 'F34',
       'F39', 'F44', 'F49', 'F54', 'F59', 'F64', 'F69', 'F74', 'F79', 'F80',
       'I04', 'I09', 'I14', 'I19', 'I24', 'I29', 'I34', 'I39', 'I44', 'I49',
       'I54', 'I59', 'I64', 'I69', 'I74', 'I79', 'I80', 'SIG', 'R01', 'R02',
       'R03', 'R04', 'R05', 'RIG', 'T01', 'T02', 'TIG', 'I01', 'I02', 'IIG']
col2 = ['contexto', 'regiao_00', 'ano', 'total_casos', 'M04', 'M09', 'M14', 'M19',
       'M24', 'M29', 'M34', 'M39', 'M44', 'M49', 'M54', 'M59', 'M64', 'M69',
       'M74', 'M79', 'M80', 'F04', 'F09', 'F14', 'F19', 'F24', 'F29', 'F34',
       'F39', 'F44', 'F49', 'F54', 'F59', 'F64', 'F69', 'F74', 'F79', 'F80',
       'I04', 'I09', 'I14', 'I19', 'I24', 'I29', 'I34', 'I39', 'I44', 'I49',
       'I54', 'I59', 'I64', 'I69', 'I74', 'I79', 'I80', 'SIG', 'R01', 'R02',
       'R03', 'R04', 'R05', 'RIG', 'T01', 'T02', 'TIG', 'I01', 'I02', 'IIG']
col3 = ['contexto', 'estado_00', 'ano', 'total_casos', 'M04', 'M09', 'M14', 'M19',
       'M24', 'M29', 'M34', 'M39', 'M44', 'M49', 'M54', 'M59', 'M64', 'M69',
       'M74', 'M79', 'M80', 'F04', 'F09', 'F14', 'F19', 'F24', 'F29', 'F34',
       'F39', 'F44', 'F49', 'F54', 'F59', 'F64', 'F69', 'F74', 'F79', 'F80',
       'I04', 'I09', 'I14', 'I19', 'I24', 'I29', 'I34', 'I39', 'I44', 'I49',
       'I54', 'I59', 'I64', 'I69', 'I74', 'I79', 'I80', 'SIG', 'R01', 'R02',
       'R03', 'R04', 'R05', 'RIG', 'T01', 'T02', 'TIG', 'I01', 'I02', 'IIG']
col4 = ['contexto', 'regiao_saude_00', 'ano', 'total_casos', 'M04', 'M09', 'M14', 'M19',
       'M24', 'M29', 'M34', 'M39', 'M44', 'M49', 'M54', 'M59', 'M64', 'M69',
       'M74', 'M79', 'M80', 'F04', 'F09', 'F14', 'F19', 'F24', 'F29', 'F34',
       'F39', 'F44', 'F49', 'F54', 'F59', 'F64', 'F69', 'F74', 'F79', 'F80',
       'I04', 'I09', 'I14', 'I19', 'I24', 'I29', 'I34', 'I39', 'I44', 'I49',
       'I54', 'I59', 'I64', 'I69', 'I74', 'I79', 'I80', 'SIG', 'R01', 'R02',
       'R03', 'R04', 'R05', 'RIG', 'T01', 'T02', 'TIG', 'I01', 'I02', 'IIG']
col5 = ['contexto', 'local_res', 'ano', 'total_casos', 'M04', 'M09', 'M14', 'M19',
       'M24', 'M29', 'M34', 'M39', 'M44', 'M49', 'M54', 'M59', 'M64', 'M69',
       'M74', 'M79', 'M80', 'F04', 'F09', 'F14', 'F19', 'F24', 'F29', 'F34',
       'F39', 'F44', 'F49', 'F54', 'F59', 'F64', 'F69', 'F74', 'F79', 'F80',
       'I04', 'I09', 'I14', 'I19', 'I24', 'I29', 'I34', 'I39', 'I44', 'I49',
       'I54', 'I59', 'I64', 'I69', 'I74', 'I79', 'I80', 'SIG', 'R01', 'R02',
       'R03', 'R04', 'R05', 'RIG', 'T01', 'T02', 'TIG', 'I01', 'I02', 'IIG']

In [31]:
df_temp = df_news[df_news['contexto_00'] != '1']
df_temp = df_temp[df_temp['local_res'] != '1']

In [32]:
#df_temp[df_temp['contexto_00'] == '1']
#df_temp[df_temp['local_res'] == 1]

In [33]:
# Gerando o agregados de ocorrencia com CID
grupo_residencia_pais = df_temp[columns].groupby([df_temp['contexto_00'], df_temp['pais'], df_temp['ano']])
grupo_residencia_regiao = df_temp[columns].groupby([df_temp['contexto_00'], df_temp['regiao_00'], df_temp['ano']])
grupo_residencia_estado = df_temp[columns].groupby([df_temp['contexto_00'], df_temp['estado_00'], df_temp['ano']])
grupo_residencia_regiao_saude = df_temp[columns].groupby([df_temp['contexto_00'], df_temp['regiao_saude_00'], df_temp['ano']])
grupo_residencia_municipio = df_temp[columns].groupby([df_temp['contexto_00'], df_temp['local_res'], df_temp['ano']])

# Gerando o agregado 0
grupo_residencia_pais_sem_contexto = df_temp[columns].groupby([df_temp['pais'], df_temp['ano']])
grupo_residencia_regiao_sem_contexto = df_temp[columns].groupby([df_temp['regiao_00'], df_temp['ano']])
grupo_residencia_estado_sem_contexto = df_temp[columns].groupby([df_temp['estado_00'], df_temp['ano']])
grupo_residencia_regiao_saude_sem_contexto = df_temp[columns].groupby([df_temp['regiao_saude_00'], df_temp['ano']])
grupo_residencia_municipio_sem_contexto = df_temp[columns].groupby([df_temp['local_res'], df_temp['ano']])

In [34]:
# Fazendo reset do index
grupo_residencia_pais = grupo_residencia_pais.sum().reset_index()
grupo_residencia_regiao = grupo_residencia_regiao.sum().reset_index()
grupo_residencia_estado = grupo_residencia_estado.sum().reset_index()
grupo_residencia_regiao_saude = grupo_residencia_regiao_saude.sum().reset_index()
grupo_residencia_municipio = grupo_residencia_municipio.sum().reset_index()

# Fazendo reset do index
grupo_residencia_pais_sem_contexto = grupo_residencia_pais_sem_contexto.sum().reset_index()
grupo_residencia_pais_sem_contexto.loc[:,'contexto'] = 0
grupo_residencia_pais_sem_contexto = grupo_residencia_pais_sem_contexto[col1]
grupo_residencia_regiao_sem_contexto = grupo_residencia_regiao_sem_contexto.sum().reset_index()
grupo_residencia_regiao_sem_contexto.loc[:,'contexto'] = 0
grupo_residencia_regiao_sem_contexto = grupo_residencia_regiao_sem_contexto[col2]
grupo_residencia_estado_sem_contexto = grupo_residencia_estado_sem_contexto.sum().reset_index()
grupo_residencia_estado_sem_contexto.loc[:,'contexto'] = 0
grupo_residencia_estado_sem_contexto = grupo_residencia_estado_sem_contexto[col3]
grupo_residencia_regiao_saude_sem_contexto = grupo_residencia_regiao_saude_sem_contexto.sum().reset_index()
grupo_residencia_regiao_saude_sem_contexto.loc[:,'contexto'] = 0
grupo_residencia_regiao_saude_sem_contexto = grupo_residencia_regiao_saude_sem_contexto[col4]
grupo_residencia_municipio_sem_contexto = grupo_residencia_municipio_sem_contexto.sum().reset_index()
grupo_residencia_municipio_sem_contexto.loc[:,'contexto'] = 0
grupo_residencia_municipio_sem_contexto = grupo_residencia_municipio_sem_contexto[col5]

In [35]:
# Renomeando as tabelas
grupo_residencia_pais.rename(columns={'pais': 'local', 'contexto_00': 'contexto'}, inplace=True)
grupo_residencia_regiao.rename(columns={'regiao_00': 'local', 'contexto_00': 'contexto'}, inplace=True)
grupo_residencia_estado.rename(columns={'estado_00': 'local', 'contexto_00': 'contexto'}, inplace=True)
grupo_residencia_regiao_saude.rename(columns={'regiao_saude_00': 'local', 'contexto_00': 'contexto'}, inplace=True)
grupo_residencia_municipio.rename(columns={'local_res': 'local', 'contexto_00': 'contexto'}, inplace=True)

grupo_residencia_pais_sem_contexto.rename(columns={'pais': 'local'}, inplace=True)
grupo_residencia_regiao_sem_contexto.rename(columns={'regiao_00': 'local'}, inplace=True)
grupo_residencia_estado_sem_contexto.rename(columns={'estado_00': 'local'}, inplace=True)
grupo_residencia_regiao_saude_sem_contexto.rename(columns={'regiao_saude_00': 'local'}, inplace=True)
grupo_residencia_municipio_sem_contexto.rename(columns={'local_res': 'local'}, inplace=True)

### Filtro ANO, LOCAL_OCO, CONTEXTO_01

In [36]:
col1 = ['contexto', 'pais', 'ano', 'total_casos', 'M04', 'M09', 'M14', 'M19',
       'M24', 'M29', 'M34', 'M39', 'M44', 'M49', 'M54', 'M59', 'M64', 'M69',
       'M74', 'M79', 'M80', 'F04', 'F09', 'F14', 'F19', 'F24', 'F29', 'F34',
       'F39', 'F44', 'F49', 'F54', 'F59', 'F64', 'F69', 'F74', 'F79', 'F80',
       'I04', 'I09', 'I14', 'I19', 'I24', 'I29', 'I34', 'I39', 'I44', 'I49',
       'I54', 'I59', 'I64', 'I69', 'I74', 'I79', 'I80', 'SIG', 'R01', 'R02',
       'R03', 'R04', 'R05', 'RIG', 'T01', 'T02', 'TIG', 'I01', 'I02', 'IIG']
col2 = ['contexto', 'regiao_01', 'ano', 'total_casos', 'M04', 'M09', 'M14', 'M19',
       'M24', 'M29', 'M34', 'M39', 'M44', 'M49', 'M54', 'M59', 'M64', 'M69',
       'M74', 'M79', 'M80', 'F04', 'F09', 'F14', 'F19', 'F24', 'F29', 'F34',
       'F39', 'F44', 'F49', 'F54', 'F59', 'F64', 'F69', 'F74', 'F79', 'F80',
       'I04', 'I09', 'I14', 'I19', 'I24', 'I29', 'I34', 'I39', 'I44', 'I49',
       'I54', 'I59', 'I64', 'I69', 'I74', 'I79', 'I80', 'SIG', 'R01', 'R02',
       'R03', 'R04', 'R05', 'RIG', 'T01', 'T02', 'TIG', 'I01', 'I02', 'IIG']
col3 = ['contexto', 'estado_01', 'ano', 'total_casos', 'M04', 'M09', 'M14', 'M19',
       'M24', 'M29', 'M34', 'M39', 'M44', 'M49', 'M54', 'M59', 'M64', 'M69',
       'M74', 'M79', 'M80', 'F04', 'F09', 'F14', 'F19', 'F24', 'F29', 'F34',
       'F39', 'F44', 'F49', 'F54', 'F59', 'F64', 'F69', 'F74', 'F79', 'F80',
       'I04', 'I09', 'I14', 'I19', 'I24', 'I29', 'I34', 'I39', 'I44', 'I49',
       'I54', 'I59', 'I64', 'I69', 'I74', 'I79', 'I80', 'SIG', 'R01', 'R02',
       'R03', 'R04', 'R05', 'RIG', 'T01', 'T02', 'TIG', 'I01', 'I02', 'IIG']
col4 = ['contexto', 'regiao_saude_01', 'ano', 'total_casos', 'M04', 'M09', 'M14', 'M19',
       'M24', 'M29', 'M34', 'M39', 'M44', 'M49', 'M54', 'M59', 'M64', 'M69',
       'M74', 'M79', 'M80', 'F04', 'F09', 'F14', 'F19', 'F24', 'F29', 'F34',
       'F39', 'F44', 'F49', 'F54', 'F59', 'F64', 'F69', 'F74', 'F79', 'F80',
       'I04', 'I09', 'I14', 'I19', 'I24', 'I29', 'I34', 'I39', 'I44', 'I49',
       'I54', 'I59', 'I64', 'I69', 'I74', 'I79', 'I80', 'SIG', 'R01', 'R02',
       'R03', 'R04', 'R05', 'RIG', 'T01', 'T02', 'TIG', 'I01', 'I02', 'IIG']
col5 = ['contexto', 'local_oco', 'ano', 'total_casos', 'M04', 'M09', 'M14', 'M19',
       'M24', 'M29', 'M34', 'M39', 'M44', 'M49', 'M54', 'M59', 'M64', 'M69',
       'M74', 'M79', 'M80', 'F04', 'F09', 'F14', 'F19', 'F24', 'F29', 'F34',
       'F39', 'F44', 'F49', 'F54', 'F59', 'F64', 'F69', 'F74', 'F79', 'F80',
       'I04', 'I09', 'I14', 'I19', 'I24', 'I29', 'I34', 'I39', 'I44', 'I49',
       'I54', 'I59', 'I64', 'I69', 'I74', 'I79', 'I80', 'SIG', 'R01', 'R02',
       'R03', 'R04', 'R05', 'RIG', 'T01', 'T02', 'TIG', 'I01', 'I02', 'IIG']

In [37]:
# Incrementando o valor de contexto para +1 para realizar o agrupamento de casos de residência (contexto XXX01)
df_temp.loc[:, 'contexto_01'] = df_news.loc[:,'contexto_01'].apply(lambda x: int(x) + 1)

df_temp = df_temp[df_news['contexto_01'] != 1]
df_temp = df_temp[df_temp['local_oco'] != '1']

/home/modesto/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [38]:
#df_temp[df_temp['contexto_01'] == '1']
#df_temp[df_temp['local_oco'] == 1]
#df_temp[df_temp['contexto_01'] == '1'].shape
#df_temp[df_temp['local_oco'] == 1].shape
#df_temp[df_news['contexto_01'] == '1']
#df_temp[df_news['local_oco'] == 1]
#df_temp[df_news['contexto_01'] == '1'].shape
#df_temp[df_news['local_oco'] == 1].shape

In [39]:
# Gerando o agregados de ocorrencia com CID
grupo_ocorrencia_pais = df_temp[columns].groupby([df_temp['contexto_01'], df_temp['pais'], df_temp['ano']])
grupo_ocorrencia_regiao = df_temp[columns].groupby([df_temp['contexto_01'], df_temp['regiao_01'], df_temp['ano']])
grupo_ocorrencia_estado = df_temp[columns].groupby([df_temp['contexto_01'], df_temp['estado_01'], df_temp['ano']])
grupo_ocorrencia_regiao_saude = df_temp[columns].groupby([df_temp['contexto_01'], df_temp['regiao_saude_01'], df_temp['ano']])
grupo_ocorrencia_municipio = df_temp[columns].groupby([df_temp['contexto_01'], df_temp['local_oco'], df_temp['ano']])

# Gerando o agregado 1
grupo_ocorrencia_pais_sem_contexto = df_temp[columns].groupby([df_temp['pais'], df_temp['ano']])
grupo_ocorrencia_regiao_sem_contexto = df_temp[columns].groupby([df_temp['regiao_01'], df_temp['ano']])
grupo_ocorrencia_estado_sem_contexto = df_temp[columns].groupby([df_temp['estado_01'], df_temp['ano']])
grupo_ocorrencia_regiao_saude_sem_contexto = df_temp[columns].groupby([df_temp['regiao_saude_01'], df_temp['ano']])
grupo_ocorrencia_municipio_sem_contexto = df_temp[columns].groupby([df_temp['local_oco'], df_temp['ano']])

In [40]:
# Fazendo reset do index
grupo_ocorrencia_pais = grupo_ocorrencia_pais.sum().reset_index()
grupo_ocorrencia_regiao = grupo_ocorrencia_regiao.sum().reset_index()
grupo_ocorrencia_estado = grupo_ocorrencia_estado.sum().reset_index()
grupo_ocorrencia_regiao_saude = grupo_ocorrencia_regiao_saude.sum().reset_index()
grupo_ocorrencia_municipio = grupo_ocorrencia_municipio.sum().reset_index()

# Fazendo reset do index
grupo_ocorrencia_pais_sem_contexto = grupo_ocorrencia_pais_sem_contexto.sum().reset_index()
grupo_ocorrencia_pais_sem_contexto.loc[:,'contexto'] = 1
grupo_ocorrencia_pais_sem_contexto = grupo_ocorrencia_pais_sem_contexto[col1]
grupo_ocorrencia_regiao_sem_contexto = grupo_ocorrencia_regiao_sem_contexto.sum().reset_index()
grupo_ocorrencia_regiao_sem_contexto.loc[:,'contexto'] = 1
grupo_ocorrencia_regiao_sem_contexto = grupo_ocorrencia_regiao_sem_contexto[col2]
grupo_ocorrencia_estado_sem_contexto = grupo_ocorrencia_estado_sem_contexto.sum().reset_index()
grupo_ocorrencia_estado_sem_contexto.loc[:,'contexto'] = 1
grupo_ocorrencia_estado_sem_contexto = grupo_ocorrencia_estado_sem_contexto[col3]
grupo_ocorrencia_regiao_saude_sem_contexto = grupo_ocorrencia_regiao_saude_sem_contexto.sum().reset_index()
grupo_ocorrencia_regiao_saude_sem_contexto.loc[:,'contexto'] = 1
grupo_ocorrencia_regiao_saude_sem_contexto = grupo_ocorrencia_regiao_saude_sem_contexto[col4]
grupo_ocorrencia_municipio_sem_contexto = grupo_ocorrencia_municipio_sem_contexto.sum().reset_index()
grupo_ocorrencia_municipio_sem_contexto.loc[:,'contexto'] = 1
grupo_ocorrencia_municipio_sem_contexto = grupo_ocorrencia_municipio_sem_contexto[col5]

In [41]:
# Renomeando as tabelas
grupo_ocorrencia_pais.rename(columns={'pais': 'local', 'contexto_01': 'contexto'}, inplace=True)
grupo_ocorrencia_regiao.rename(columns={'regiao_01': 'local', 'contexto_01': 'contexto'}, inplace=True)
grupo_ocorrencia_estado.rename(columns={'estado_01': 'local', 'contexto_01': 'contexto'}, inplace=True)
grupo_ocorrencia_regiao_saude.rename(columns={'regiao_saude_01': 'local', 'contexto_01': 'contexto'}, inplace=True)
grupo_ocorrencia_municipio.rename(columns={'local_oco': 'local', 'contexto_01': 'contexto'}, inplace=True)

grupo_ocorrencia_pais_sem_contexto.rename(columns={'pais': 'local'}, inplace=True)
grupo_ocorrencia_regiao_sem_contexto.rename(columns={'regiao_01': 'local'}, inplace=True)
grupo_ocorrencia_estado_sem_contexto.rename(columns={'estado_01': 'local'}, inplace=True)
grupo_ocorrencia_regiao_saude_sem_contexto.rename(columns={'regiao_saude_01': 'local'}, inplace=True)
grupo_ocorrencia_municipio_sem_contexto.rename(columns={'local_oco': 'local'}, inplace=True)

In [42]:
concatenar = [
    grupo_residencia_pais,
    grupo_residencia_regiao,
    grupo_residencia_estado,
    grupo_residencia_regiao_saude,
    grupo_residencia_municipio,
    grupo_residencia_pais_sem_contexto,
    grupo_residencia_regiao_sem_contexto,
    grupo_residencia_estado_sem_contexto,
    grupo_residencia_regiao_saude_sem_contexto,
    grupo_residencia_municipio_sem_contexto,
    grupo_ocorrencia_pais,
    grupo_ocorrencia_regiao,
    grupo_ocorrencia_estado,
    grupo_ocorrencia_regiao_saude,
    grupo_ocorrencia_municipio,
    grupo_ocorrencia_pais_sem_contexto,
    grupo_ocorrencia_regiao_sem_contexto,
    grupo_ocorrencia_estado_sem_contexto,
    grupo_ocorrencia_regiao_saude_sem_contexto,
    grupo_ocorrencia_municipio_sem_contexto,
]

In [43]:
df_final = pd.concat(concatenar)

In [44]:
# GERAR CSV
df_final.to_csv('consolidacao/consolidacao_ANO_06.csv', index=False, header=False)

In [45]:
# Renomear (resetar) os index
# Adicionar coluna do 0 e 1 aos sem contexto
# Reordenar index
# Gerar csv

### Filtro SEMANA, LOCAL_RES, CONTEXTO_00

In [46]:
col1 = ['contexto', 'pais', 'ano_sem', 'semana', 'total_casos', 'M04', 'M09', 'M14', 'M19',
       'M24', 'M29', 'M34', 'M39', 'M44', 'M49', 'M54', 'M59', 'M64', 'M69',
       'M74', 'M79', 'M80', 'F04', 'F09', 'F14', 'F19', 'F24', 'F29', 'F34',
       'F39', 'F44', 'F49', 'F54', 'F59', 'F64', 'F69', 'F74', 'F79', 'F80',
       'I04', 'I09', 'I14', 'I19', 'I24', 'I29', 'I34', 'I39', 'I44', 'I49',
       'I54', 'I59', 'I64', 'I69', 'I74', 'I79', 'I80', 'SIG', 'R01', 'R02',
       'R03', 'R04', 'R05', 'RIG', 'T01', 'T02', 'TIG', 'I01', 'I02', 'IIG']
col2 = ['contexto', 'regiao_00', 'ano_sem', 'semana', 'total_casos', 'M04', 'M09', 'M14', 'M19',
       'M24', 'M29', 'M34', 'M39', 'M44', 'M49', 'M54', 'M59', 'M64', 'M69',
       'M74', 'M79', 'M80', 'F04', 'F09', 'F14', 'F19', 'F24', 'F29', 'F34',
       'F39', 'F44', 'F49', 'F54', 'F59', 'F64', 'F69', 'F74', 'F79', 'F80',
       'I04', 'I09', 'I14', 'I19', 'I24', 'I29', 'I34', 'I39', 'I44', 'I49',
       'I54', 'I59', 'I64', 'I69', 'I74', 'I79', 'I80', 'SIG', 'R01', 'R02',
       'R03', 'R04', 'R05', 'RIG', 'T01', 'T02', 'TIG', 'I01', 'I02', 'IIG']
col3 = ['contexto', 'estado_00', 'ano_sem', 'semana', 'total_casos', 'M04', 'M09', 'M14', 'M19',
       'M24', 'M29', 'M34', 'M39', 'M44', 'M49', 'M54', 'M59', 'M64', 'M69',
       'M74', 'M79', 'M80', 'F04', 'F09', 'F14', 'F19', 'F24', 'F29', 'F34',
       'F39', 'F44', 'F49', 'F54', 'F59', 'F64', 'F69', 'F74', 'F79', 'F80',
       'I04', 'I09', 'I14', 'I19', 'I24', 'I29', 'I34', 'I39', 'I44', 'I49',
       'I54', 'I59', 'I64', 'I69', 'I74', 'I79', 'I80', 'SIG', 'R01', 'R02',
       'R03', 'R04', 'R05', 'RIG', 'T01', 'T02', 'TIG', 'I01', 'I02', 'IIG']
col4 = ['contexto', 'regiao_saude_00', 'ano_sem', 'semana', 'total_casos', 'M04', 'M09', 'M14', 'M19',
       'M24', 'M29', 'M34', 'M39', 'M44', 'M49', 'M54', 'M59', 'M64', 'M69',
       'M74', 'M79', 'M80', 'F04', 'F09', 'F14', 'F19', 'F24', 'F29', 'F34',
       'F39', 'F44', 'F49', 'F54', 'F59', 'F64', 'F69', 'F74', 'F79', 'F80',
       'I04', 'I09', 'I14', 'I19', 'I24', 'I29', 'I34', 'I39', 'I44', 'I49',
       'I54', 'I59', 'I64', 'I69', 'I74', 'I79', 'I80', 'SIG', 'R01', 'R02',
       'R03', 'R04', 'R05', 'RIG', 'T01', 'T02', 'TIG', 'I01', 'I02', 'IIG']
col5 = ['contexto', 'local_res', 'ano_sem', 'semana', 'total_casos', 'M04', 'M09', 'M14', 'M19',
       'M24', 'M29', 'M34', 'M39', 'M44', 'M49', 'M54', 'M59', 'M64', 'M69',
       'M74', 'M79', 'M80', 'F04', 'F09', 'F14', 'F19', 'F24', 'F29', 'F34',
       'F39', 'F44', 'F49', 'F54', 'F59', 'F64', 'F69', 'F74', 'F79', 'F80',
       'I04', 'I09', 'I14', 'I19', 'I24', 'I29', 'I34', 'I39', 'I44', 'I49',
       'I54', 'I59', 'I64', 'I69', 'I74', 'I79', 'I80', 'SIG', 'R01', 'R02',
       'R03', 'R04', 'R05', 'RIG', 'T01', 'T02', 'TIG', 'I01', 'I02', 'IIG']

In [47]:
df_temp = df_news[df_news['contexto_00'] != '1']
df_temp = df_temp[df_temp['local_res'] != '1']
df_temp = df_temp[df_temp['ano_sem'] != '1000']
df_temp = df_temp[df_temp['semana'] != '1000']

In [48]:
#df_temp[df_temp['contexto_00'] == '1']
#df_temp[df_temp['local_res'] == '1']
#df_temp[df_temp['ano_sem'] == 1000]
#df_temp[df_temp['semana'] == 1000]

In [49]:
# Gerando o agregados de ocorrencia com CID
grupo_residencia_pais = df_temp[columns].groupby([df_temp['contexto_00'], df_temp['pais'], df_temp['ano_sem'], df_temp['semana']])
grupo_residencia_regiao = df_temp[columns].groupby([df_temp['contexto_00'], df_temp['regiao_00'], df_temp['ano_sem'], df_temp['semana']])
grupo_residencia_estado = df_temp[columns].groupby([df_temp['contexto_00'], df_temp['estado_00'], df_temp['ano_sem'], df_temp['semana']])
grupo_residencia_regiao_saude = df_temp[columns].groupby([df_temp['contexto_00'], df_temp['regiao_saude_00'], df_temp['ano_sem'], df_temp['semana']])
grupo_residencia_municipio = df_temp[columns].groupby([df_temp['contexto_00'], df_temp['local_res'], df_temp['ano_sem'], df_temp['semana']])

# Gerando o agregado 0
grupo_residencia_pais_sem_contexto = df_temp[columns].groupby([df_temp['pais'], df_temp['ano_sem'], df_temp['semana']])
grupo_residencia_regiao_sem_contexto = df_temp[columns].groupby([df_temp['regiao_00'], df_temp['ano_sem'], df_temp['semana']])
grupo_residencia_estado_sem_contexto = df_temp[columns].groupby([df_temp['estado_00'], df_temp['ano_sem'], df_temp['semana']])
grupo_residencia_regiao_saude_sem_contexto = df_temp[columns].groupby([df_temp['regiao_saude_00'], df_temp['ano_sem'], df_temp['semana']])
grupo_residencia_municipio_sem_contexto = df_temp[columns].groupby([df_temp['local_res'], df_temp['ano_sem'], df_temp['semana']])

In [50]:
# Fazendo reset do index
grupo_residencia_pais = grupo_residencia_pais.sum().reset_index()
grupo_residencia_regiao = grupo_residencia_regiao.sum().reset_index()
grupo_residencia_estado = grupo_residencia_estado.sum().reset_index()
grupo_residencia_regiao_saude = grupo_residencia_regiao_saude.sum().reset_index()
grupo_residencia_municipio = grupo_residencia_municipio.sum().reset_index()

# Fazendo reset do index
grupo_residencia_pais_sem_contexto = grupo_residencia_pais_sem_contexto.sum().reset_index()
grupo_residencia_pais_sem_contexto.loc[:,'contexto'] = 0
grupo_residencia_pais_sem_contexto = grupo_residencia_pais_sem_contexto[col1]
grupo_residencia_regiao_sem_contexto = grupo_residencia_regiao_sem_contexto.sum().reset_index()
grupo_residencia_regiao_sem_contexto.loc[:,'contexto'] = 0
grupo_residencia_regiao_sem_contexto = grupo_residencia_regiao_sem_contexto[col2]
grupo_residencia_estado_sem_contexto = grupo_residencia_estado_sem_contexto.sum().reset_index()
grupo_residencia_estado_sem_contexto.loc[:,'contexto'] = 0
grupo_residencia_estado_sem_contexto = grupo_residencia_estado_sem_contexto[col3]
grupo_residencia_regiao_saude_sem_contexto = grupo_residencia_regiao_saude_sem_contexto.sum().reset_index()
grupo_residencia_regiao_saude_sem_contexto.loc[:,'contexto'] = 0
grupo_residencia_regiao_saude_sem_contexto = grupo_residencia_regiao_saude_sem_contexto[col4]
grupo_residencia_municipio_sem_contexto = grupo_residencia_municipio_sem_contexto.sum().reset_index()
grupo_residencia_municipio_sem_contexto.loc[:,'contexto'] = 0
grupo_residencia_municipio_sem_contexto = grupo_residencia_municipio_sem_contexto[col5]

In [51]:
# Renomeando as tabelas
grupo_residencia_pais.rename(columns={'pais': 'local', 'contexto_00': 'contexto'}, inplace=True)
grupo_residencia_regiao.rename(columns={'regiao_00': 'local', 'contexto_00': 'contexto'}, inplace=True)
grupo_residencia_estado.rename(columns={'estado_00': 'local', 'contexto_00': 'contexto'}, inplace=True)
grupo_residencia_regiao_saude.rename(columns={'regiao_saude_00': 'local', 'contexto_00': 'contexto'}, inplace=True)
grupo_residencia_municipio.rename(columns={'local_res': 'local', 'contexto_00': 'contexto'}, inplace=True)

grupo_residencia_pais_sem_contexto.rename(columns={'pais': 'local'}, inplace=True)
grupo_residencia_regiao_sem_contexto.rename(columns={'regiao_00': 'local'}, inplace=True)
grupo_residencia_estado_sem_contexto.rename(columns={'estado_00': 'local'}, inplace=True)
grupo_residencia_regiao_saude_sem_contexto.rename(columns={'regiao_saude_00': 'local'}, inplace=True)
grupo_residencia_municipio_sem_contexto.rename(columns={'local_res': 'local'}, inplace=True)

### Filtro SEMANA, LOCAL_OCO, CONTEXTO_01

In [52]:
col1 = ['contexto', 'pais', 'ano_sem', 'semana', 'total_casos', 'M04', 'M09', 'M14', 'M19',
       'M24', 'M29', 'M34', 'M39', 'M44', 'M49', 'M54', 'M59', 'M64', 'M69',
       'M74', 'M79', 'M80', 'F04', 'F09', 'F14', 'F19', 'F24', 'F29', 'F34',
       'F39', 'F44', 'F49', 'F54', 'F59', 'F64', 'F69', 'F74', 'F79', 'F80',
       'I04', 'I09', 'I14', 'I19', 'I24', 'I29', 'I34', 'I39', 'I44', 'I49',
       'I54', 'I59', 'I64', 'I69', 'I74', 'I79', 'I80', 'SIG', 'R01', 'R02',
       'R03', 'R04', 'R05', 'RIG', 'T01', 'T02', 'TIG', 'I01', 'I02', 'IIG']
col2 = ['contexto', 'regiao_01', 'ano_sem', 'semana', 'total_casos', 'M04', 'M09', 'M14', 'M19',
       'M24', 'M29', 'M34', 'M39', 'M44', 'M49', 'M54', 'M59', 'M64', 'M69',
       'M74', 'M79', 'M80', 'F04', 'F09', 'F14', 'F19', 'F24', 'F29', 'F34',
       'F39', 'F44', 'F49', 'F54', 'F59', 'F64', 'F69', 'F74', 'F79', 'F80',
       'I04', 'I09', 'I14', 'I19', 'I24', 'I29', 'I34', 'I39', 'I44', 'I49',
       'I54', 'I59', 'I64', 'I69', 'I74', 'I79', 'I80', 'SIG', 'R01', 'R02',
       'R03', 'R04', 'R05', 'RIG', 'T01', 'T02', 'TIG', 'I01', 'I02', 'IIG']
col3 = ['contexto', 'estado_01', 'ano_sem', 'semana', 'total_casos', 'M04', 'M09', 'M14', 'M19',
       'M24', 'M29', 'M34', 'M39', 'M44', 'M49', 'M54', 'M59', 'M64', 'M69',
       'M74', 'M79', 'M80', 'F04', 'F09', 'F14', 'F19', 'F24', 'F29', 'F34',
       'F39', 'F44', 'F49', 'F54', 'F59', 'F64', 'F69', 'F74', 'F79', 'F80',
       'I04', 'I09', 'I14', 'I19', 'I24', 'I29', 'I34', 'I39', 'I44', 'I49',
       'I54', 'I59', 'I64', 'I69', 'I74', 'I79', 'I80', 'SIG', 'R01', 'R02',
       'R03', 'R04', 'R05', 'RIG', 'T01', 'T02', 'TIG', 'I01', 'I02', 'IIG']
col4 = ['contexto', 'regiao_saude_01', 'ano_sem', 'semana', 'total_casos', 'M04', 'M09', 'M14', 'M19',
       'M24', 'M29', 'M34', 'M39', 'M44', 'M49', 'M54', 'M59', 'M64', 'M69',
       'M74', 'M79', 'M80', 'F04', 'F09', 'F14', 'F19', 'F24', 'F29', 'F34',
       'F39', 'F44', 'F49', 'F54', 'F59', 'F64', 'F69', 'F74', 'F79', 'F80',
       'I04', 'I09', 'I14', 'I19', 'I24', 'I29', 'I34', 'I39', 'I44', 'I49',
       'I54', 'I59', 'I64', 'I69', 'I74', 'I79', 'I80', 'SIG', 'R01', 'R02',
       'R03', 'R04', 'R05', 'RIG', 'T01', 'T02', 'TIG', 'I01', 'I02', 'IIG']
col5 = ['contexto', 'local_oco', 'ano_sem', 'semana', 'total_casos', 'M04', 'M09', 'M14', 'M19',
       'M24', 'M29', 'M34', 'M39', 'M44', 'M49', 'M54', 'M59', 'M64', 'M69',
       'M74', 'M79', 'M80', 'F04', 'F09', 'F14', 'F19', 'F24', 'F29', 'F34',
       'F39', 'F44', 'F49', 'F54', 'F59', 'F64', 'F69', 'F74', 'F79', 'F80',
       'I04', 'I09', 'I14', 'I19', 'I24', 'I29', 'I34', 'I39', 'I44', 'I49',
       'I54', 'I59', 'I64', 'I69', 'I74', 'I79', 'I80', 'SIG', 'R01', 'R02',
       'R03', 'R04', 'R05', 'RIG', 'T01', 'T02', 'TIG', 'I01', 'I02', 'IIG']

In [53]:
# Incrementando o valor de contexto para +1 para realizar o agrupamento de casos de residência (contexto XXX01)
df_temp.loc[:, 'contexto_01'] = df_news.loc[:,'contexto_01'].apply(lambda x: int(x) + 1)

df_temp = df_temp[df_news['contexto_01'] != 1]
df_temp = df_temp[df_temp['local_oco'] != '1']
df_temp = df_temp[df_temp['ano_sem'] != '1000']
df_temp = df_temp[df_temp['semana'] != '1000']

/home/modesto/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [54]:
# df_temp = df_news[df_news['contexto_01'] != '1']
# df_temp = df_temp[df_temp['local_oco'] != '1']
# df_temp = df_temp[df_temp['ano_sem'] != '1000']
# df_temp = df_temp[df_temp['semana'] != '1000']

In [55]:
# Gerando o agregados de ocorrencia com CID
grupo_ocorrencia_pais = df_temp[columns].groupby([df_temp['contexto_01'], df_temp['pais'], df_temp['ano_sem'], df_temp['semana']])
grupo_ocorrencia_regiao = df_temp[columns].groupby([df_temp['contexto_01'], df_temp['regiao_01'], df_temp['ano_sem'], df_temp['semana']])
grupo_ocorrencia_estado = df_temp[columns].groupby([df_temp['contexto_01'], df_temp['estado_01'], df_temp['ano_sem'], df_temp['semana']])
grupo_ocorrencia_regiao_saude = df_temp[columns].groupby([df_temp['contexto_01'], df_temp['regiao_saude_01'], df_temp['ano_sem'], df_temp['semana']])
grupo_ocorrencia_municipio = df_temp[columns].groupby([df_temp['contexto_01'], df_temp['local_oco'], df_temp['ano_sem'], df_temp['semana']])

# Gerando o agregado 1
grupo_ocorrencia_pais_sem_contexto = df_temp[columns].groupby([df_temp['pais'], df_temp['ano_sem'], df_temp['semana']])
grupo_ocorrencia_regiao_sem_contexto = df_temp[columns].groupby([df_temp['regiao_01'], df_temp['ano_sem'], df_temp['semana']])
grupo_ocorrencia_estado_sem_contexto = df_temp[columns].groupby([df_temp['estado_01'], df_temp['ano_sem'], df_temp['semana']])
grupo_ocorrencia_regiao_saude_sem_contexto = df_temp[columns].groupby([df_temp['regiao_saude_01'], df_temp['ano_sem'], df_temp['semana']])
grupo_ocorrencia_municipio_sem_contexto = df_temp[columns].groupby([df_temp['local_oco'], df_temp['ano_sem'], df_temp['semana']])

In [56]:
# Fazendo reset do index
grupo_ocorrencia_pais = grupo_ocorrencia_pais.sum().reset_index()
grupo_ocorrencia_regiao = grupo_ocorrencia_regiao.sum().reset_index()
grupo_ocorrencia_estado = grupo_ocorrencia_estado.sum().reset_index()
grupo_ocorrencia_regiao_saude = grupo_ocorrencia_regiao_saude.sum().reset_index()
grupo_ocorrencia_municipio = grupo_ocorrencia_municipio.sum().reset_index()

# Fazendo reset do index
grupo_ocorrencia_pais_sem_contexto = grupo_ocorrencia_pais_sem_contexto.sum().reset_index()
grupo_ocorrencia_pais_sem_contexto.loc[:,'contexto'] = 1
grupo_ocorrencia_pais_sem_contexto = grupo_ocorrencia_pais_sem_contexto[col1]
grupo_ocorrencia_regiao_sem_contexto = grupo_ocorrencia_regiao_sem_contexto.sum().reset_index()
grupo_ocorrencia_regiao_sem_contexto.loc[:,'contexto'] = 1
grupo_ocorrencia_regiao_sem_contexto = grupo_ocorrencia_regiao_sem_contexto[col2]
grupo_ocorrencia_estado_sem_contexto = grupo_ocorrencia_estado_sem_contexto.sum().reset_index()
grupo_ocorrencia_estado_sem_contexto.loc[:,'contexto'] = 1
grupo_ocorrencia_estado_sem_contexto = grupo_ocorrencia_estado_sem_contexto[col3]
grupo_ocorrencia_regiao_saude_sem_contexto = grupo_ocorrencia_regiao_saude_sem_contexto.sum().reset_index()
grupo_ocorrencia_regiao_saude_sem_contexto.loc[:,'contexto'] = 1
grupo_ocorrencia_regiao_saude_sem_contexto = grupo_ocorrencia_regiao_saude_sem_contexto[col4]
grupo_ocorrencia_municipio_sem_contexto = grupo_ocorrencia_municipio_sem_contexto.sum().reset_index()
grupo_ocorrencia_municipio_sem_contexto.loc[:,'contexto'] = 1
grupo_ocorrencia_municipio_sem_contexto = grupo_ocorrencia_municipio_sem_contexto[col5]

In [57]:
# Renomeando as tabelas
grupo_ocorrencia_pais.rename(columns={'pais': 'local', 'contexto_01': 'contexto'}, inplace=True)
grupo_ocorrencia_regiao.rename(columns={'regiao_01': 'local', 'contexto_01': 'contexto'}, inplace=True)
grupo_ocorrencia_estado.rename(columns={'estado_01': 'local', 'contexto_01': 'contexto'}, inplace=True)
grupo_ocorrencia_regiao_saude.rename(columns={'regiao_saude_01': 'local', 'contexto_01': 'contexto'}, inplace=True)
grupo_ocorrencia_municipio.rename(columns={'local_oco': 'local', 'contexto_01': 'contexto'}, inplace=True)

grupo_ocorrencia_pais_sem_contexto.rename(columns={'pais': 'local'}, inplace=True)
grupo_ocorrencia_regiao_sem_contexto.rename(columns={'regiao_01': 'local'}, inplace=True)
grupo_ocorrencia_estado_sem_contexto.rename(columns={'estado_01': 'local'}, inplace=True)
grupo_ocorrencia_regiao_saude_sem_contexto.rename(columns={'regiao_saude_01': 'local'}, inplace=True)
grupo_ocorrencia_municipio_sem_contexto.rename(columns={'local_oco': 'local'}, inplace=True)

In [58]:
concatenar = [
    grupo_residencia_pais,
    grupo_residencia_regiao,
    grupo_residencia_estado,
    grupo_residencia_regiao_saude,
    grupo_residencia_municipio,
    grupo_residencia_pais_sem_contexto,
    grupo_residencia_regiao_sem_contexto,
    grupo_residencia_estado_sem_contexto,
    grupo_residencia_regiao_saude_sem_contexto,
    grupo_residencia_municipio_sem_contexto,
    grupo_ocorrencia_pais,
    grupo_ocorrencia_regiao,
    grupo_ocorrencia_estado,
    grupo_ocorrencia_regiao_saude,
    grupo_ocorrencia_municipio,
    grupo_ocorrencia_pais_sem_contexto,
    grupo_ocorrencia_regiao_sem_contexto,
    grupo_ocorrencia_estado_sem_contexto,
    grupo_ocorrencia_regiao_saude_sem_contexto,
    grupo_ocorrencia_municipio_sem_contexto,
]

In [59]:
df_final = pd.concat(concatenar)

In [60]:
df_final.to_csv('consolidacao/consolidacao_SEMANA_06.csv', index=False, header=False)